In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import pandas as pd
import warnings
import os
import sys
import re

sys.path.append("../")
warnings.filterwarnings("ignore")

In [2]:
import os


root_dir = os.path.abspath('../bailian_nlp/')

data_path = os.path.join(root_dir, 'datadrive/bailian/pos')
if not os.path.exists(data_path):
    os.makedirs(data_path, exist_ok=True)

train_path = os.path.join(data_path, 'train.csv')
valid_path = os.path.join(data_path, 'valid.csv')

model_dir = os.path.join(root_dir, 'datadrive/models/chinese_L-12_H-768_A-12/')
init_checkpoint_pt = os.path.join(model_dir, 'bert_model.bin')
bert_config_file = os.path.join(model_dir, 'bert_config.json')
vocab_file = os.path.join(model_dir, 'vocab.txt')
model_pt = os.path.join(model_dir, 'pos.bin')
config_file = os.path.join(model_dir, 'pos.json')



In [ ]:
# 初始化构造数据
from bailian_nlp.preprocessing.build_data import build_pos_fake_data, build_pos_train_and_valid_data
# build_pos_fake_data()
build_pos_train_and_valid_data()



In [ ]:
# 正常训练

from bailian_nlp.modules import BertData

data = BertData.create(
    train_path,
    valid_path, 
    vocab_file,
    data_type="bert_uncased",
    max_seq_len=128,
    batch_size=128
    
)

import torch
import torch.nn as nn
from importlib import reload
from bailian_nlp.modules.models import bert_models
reload(bert_models)

model = bert_models.BertBiLSTMAttnCRF.create(
    len(data.label2idx),
    enc_hidden_dim=256
)


from bailian_nlp.modules.train import train
reload(train)
num_epochs = 10
learner = train.NerLearner(model, data,
                     best_model_path=model_pt,
                     lr=0.001, clip=1.0, sup_labels=data.id2label,
                     t_total=num_epochs * len(data.train_dl))

learner.fit(num_epochs, target_metric='f1')


In [ ]:
import torch
import torch.nn as nn
from importlib import reload
from bailian_nlp.modules.models import bert_models
reload(bert_models)

model = bert_models.BertBiLSTMAttnCRF.create(
    len(data.label2idx),
    enc_hidden_dim=256
)
model.get_n_trainable_params()


from bailian_nlp.modules.train import train
reload(train)
num_epochs = 2
learner = train.NerLearner(model, data,
                     best_model_path=model_pt,
                     lr=0.001, clip=1.0, sup_labels=data.id2label,
                     t_total=num_epochs * len(data.train_dl))

learner.fit(num_epochs, target_metric='f1')

In [ ]:
# 恢复训练

from bailian_nlp.released import pos
from importlib import reload
reload(pos)
tagger = pos.PosTagger()
tagger.init_env(for_train=True)

data = tagger.learner.data
learner = tagger.learner
num_epochs = 1
learner.load_model()
learner.t_total = num_epochs * len(data.train_dl)
learner.sup_labels = list(set(data.id2label[1:]) | set(learner.sup_labels))
learner.fit(num_epochs, target_metric='f1')


In [ ]:
learner.fit(num_epochs, target_metric='f1')

In [ ]:
from modules.data import bert_data
reload(bert_data)
dl = bert_data.get_bert_data_loader_for_predict(valid_path, learner)

In [ ]:
# learner.load_model()
preds = learner.predict(dl)

In [ ]:
from modules.utils.plot_metrics import get_bert_span_report
clf_report = get_bert_span_report(dl, preds)
print(clf_report)

In [ ]:
from bailian_nlp.released import pos
from importlib import reload
reload(pos)
tagger = pos.PosTagger()


In [ ]:
import time

st = time.time()
text = '近日，编程猫（深圳点猫科技有限公司）正式对外宣布完成B轮1.2亿元融资。本轮融资由高瓴资本领投，清流资本、清晗基金跟投，天使轮投资者猎豹移动继续跟投。'
# text = '未来编程教育产业将蓬勃发展，编程猫作为提供工具与内容的企业，有望长期处于行业领跑者地位。'
# text = '美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司美年大健康产业（集团）有限公司始创于2004年,是中国健康体检和医疗服务集团,总部位于上海,深耕布局北京、深圳、沈阳、广州、成都、武汉、...'
# text = '百炼智能百炼智能'
# text = '高越君冯是聪'
text = '周光明确否认CEO佟显侨和衡量推动发出公司公告'
text = '周光明确否认CEO佟显侨和CTO衡量说的罪名'
text = '董事'
# text = '一言九鼎'
# text = '客户包括雀巢、洲际酒店、瑞士航空、德意志银行、红牛、瑞士联合银行等世界知名公司。'
# text = '药方越是多的，越表明病是难的于治疗'
text = '2012-11-8 的客户包括：宝洁 、杜邦、强生、百事可 乐、3M 、三菱、吉列、通用医疗系统、美国 运通、雅芳、纽约银 行、百时美—施贵 宝、礼来、迪斯尼、纳贝斯克、纽约'
res = tagger.cut(text)
ed = time.time()
print(ed - st)
res

In [24]:
a = 'á'
acc = a[1]

In [67]:
tokenizer.tokenize(ww)

(['1',
  '.',
  '未',
  'a',
  '来',
  '伙',
  '伴',
  '机',
  '器',
  '人',
  '费',
  '旭',
  '锋',
  '：',
  '教',
  '育',
  '机',
  '器',
  '人',
  '市',
  '场',
  '预',
  '计',
  '在',
  '2025',
  '年',
  '会',
  '达',
  '到',
  '3000',
  '亿'],
 [(-2, (1, 2)),
  (-2, (3, 4)),
  (-2, (4, 5)),
  (1, (0, 1)),
  (1, (1, 2)),
  (1, (5, 6)),
  (-2, (7, 8)),
  (1, (6, 7)),
  (1, (8, 9)),
  (-2, (9, 10)),
  (-2, (10, 11)),
  (1, (11, 12)),
  (1, (12, 13)),
  (1, (13, 14)),
  (1, (14, 15)),
  (1, (15, 16)),
  (1, (16, 17)),
  (1, (17, 18)),
  (1, (18, 19)),
  (1, (19, 20)),
  (0, (20, 21)),
  (1, (21, 22)),
  (1, (22, 23)),
  (1, (23, 24)),
  (1, (24, 25)),
  (1, (25, 26)),
  (1, (26, 27)),
  (1, (27, 28)),
  (1, (28, 29)),
  (1, (29, 30)),
  (1, (30, 31)),
  (1, (31, 35)),
  (1, (35, 36)),
  (1, (36, 37)),
  (1, (37, 38)),
  (1, (38, 39)),
  (1, (39, 43)),
  (1, (43, 44))])

In [66]:
ww = '1́.́́未á来́́伙伴机器人费旭锋： 教育机器人市场预计在2025年会达到3000亿'
ww[5]

'未'

In [64]:
from bailian_nlp.modules.data.tokenization import BailianTokenizer
tokenizer = BailianTokenizer()
text = f'1{acc}/m ./w {acc}{acc}未á来{acc}{acc}伙伴机器人/nt 费旭锋/nr ：/w  /w 教育机器人/nz 市场/n 预计/v 在/p 2025/m 年会/n 达到/v 3000亿/m'
tokenizer.tokenize_with_pos_text(text)

[(-2, (1, 2)), (-2, (3, 4)), (-2, (4, 5)), (1, (0, 1)), (1, (1, 2)), (1, (5, 6)), (-2, (7, 8)), (1, (6, 7)), (1, (8, 9)), (-2, (9, 10)), (-2, (10, 11)), (1, (11, 12)), (1, (12, 13)), (1, (13, 14)), (1, (14, 15)), (1, (15, 16)), (1, (16, 17)), (1, (17, 18)), (1, (18, 19)), (1, (19, 20)), (0, (20, 21)), (1, (21, 22)), (1, (22, 23)), (1, (23, 24)), (1, (24, 25)), (1, (25, 26)), (1, (26, 27)), (1, (27, 28)), (1, (28, 29)), (1, (29, 30)), (1, (30, 31)), (1, (31, 35)), (1, (35, 36)), (1, (36, 37)), (1, (37, 38)), (1, (38, 39)), (1, (39, 43)), (1, (43, 44))]
1́.́́未á来́́伙伴机器人费旭锋： 教育机器人市场预计在2025年会达到3000亿
99999 1 0 1
ps 1 0 0 0
wwwwww 1 [('m', 0)] 1 0
99999 1 1 2
eneeeeeee 1 2
ps ́. 1 0 0
wwwwww 2 [('m', 0), ('w', 1)] ́. 0
99999 1 5 6
ps  5 1 0
wwwwww 3 []  0
99999 1 6 7
ps a 6 0 0
wwwwww 4 [('nt', 2)] a 0
99999 1 8 9
ps 来 8 0 0
wwwwww 5 [('nt', 2)] 来 0
99999 1 11 12
ps 伙 11 0 0
wwwwww 6 [('nt', 2)] 伙 0
99999 1 12 13
ps 伴 12 0 0
wwwwww 7 [('nt', 2)] 伴 0
99999 1 13 14
ps 机 13 0 0
wwwwww 8 [('nt',

Exception: 原有分词与现有tokenizer有冲突

In [20]:
a[6]

'/'